In [1]:
import warnings
warnings.filterwarnings('ignore')
import re
import pickle
import numpy as np
import pandas as pd
import pickle
import operator
from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
def main():
    
    #Read pre-processed train_paras.txt file into codes
    
    with open('train_paras.txt','r') as f:
        codes=f.readlines()
    
    for i in range(0,len(codes)):
        codes[i]=codes[i][2:].split("', '")
        codes[i][-1]=codes[i][-1].replace("']\n",'')
    
    #Read tags.pkl as tags
    with open("tags.pkl", "rb") as fp:
        tags = pickle.load(fp)
        
    #for i in range(0,len(tags)):
        #print(tags[i])
        #tags[i]=tags[i][0]
        
    for i in range(0,len(codes)):
        codes[i]=" ".join(word for word in codes[i])
    
    #Call code_classification
    code_classification(codes,tags)

In [6]:
df = pd.read_csv('MyData.csv')

def all_label(raw_data):
    l = len(raw_data)
    label_list = [] #np.empty((raw_data.shape[0],len(tagCol)))
    ll =[]
    tagCol = [col for col in raw_data.columns if 'Tag_' in col]
    #print(tagCol)
    for j in range(0,raw_data.shape[0]):
        for i in range(0,len(tagCol)):
            label_list.append(tagCol[i])
        #print(label_list)
    return label_list

label_entire = all_label(df)
label_entire = list(dict.fromkeys(label_entire)) 
#label_entire

x = []
for i in range(0,len(label_entire)):
    x.append(df[label_entire[i]].sum())

SelectMax= pd.DataFrame()
SelectMax['Tag']=label_entire
SelectMax['Count']=x

SelectMax = SelectMax.sort_values(by='Count', ascending=False)
#print(SelectMax)

In [12]:
#print(SelectMax.loc[1:9,:])
def code_classification(codes,tags):
    
    # Chose top tags to train into two different model. Made box plots to look for outliers
    
    #Top 9 most frequent tags
    new_tags1=SelectMax.iloc[1:9,:]
    new_tags1 = new_tags1['Tag'].tolist()
    #print(new_tags1)
    #print(type(new_tags1))
    
    #print(type(tags))
    
    #Next 11 most frequent tags
    new_tags2=SelectMax.iloc[10:20,:]
    #new_tags2 = np.asarray(new_tags2)
    new_tags2 = new_tags2['Tag'].tolist()
    
    #Indices of specified frequent tags in new_tags1, new_tags2
    indexes_1=[]
    indexes_2=[]
    for i in range(len(tags)):
        for j in range(len(tags[i])):
            #print(tags[i][j])
            if tags[i][j] in new_tags1:
                indexes_1.append(i)
            elif tags[i][j] in new_tags2:
                indexes_2.append(i)

    #Separating frequent tags to perform classification to maintain class balance
    new_codes1=operator.itemgetter(*indexes_1)(codes)
    tags1=operator.itemgetter(*indexes_1)(tags)
    
    #Model building for top 9 frequent tags
    vectorizer,model=model_building(new_codes1,tags1)
    
    #Saving the vectorizer and model
    with open('vectorizer_1.pkl', 'wb') as f:
        pickle.dump(vectorizer, f)
    with open('model_codes_1.pkl', 'wb') as f:
        pickle.dump(model, f)
    
    #Separating next most frequent tags to perform classification to maintain class balance
    new_codes2=operator.itemgetter(*indexes_2)(codes)
    tags2=operator.itemgetter(*indexes_2)(tags)
    
    #Building model for next 11 frequent tags
    vectorizer,model=model_building(new_codes2,tags2)
    
    #Saving model and vectorizer
    with open('vectorizer_2.pkl', 'wb') as f:
        pickle.dump(vectorizer, f)
    with open('model_codes_2.pkl', 'wb') as f:
        pickle.dump(model, f)

In [8]:
def model_building(new_codes,tags):
    
    # Use TF-IDF vectorizer and extract features to give it as an inout to the LogisticRegression Model
    
    
    #Building TF-IDF
    vectorizer=TfidfVectorizer(analyzer='word',stop_words='english')
    transformed=vectorizer.fit_transform(new_codes)
    
    #Building Logistic regression to classify codes
    model=LogisticRegression()
    model.fit(transformed,tags)
    
    #Return model and vectorizer
    return vectorizer, model

In [13]:
if __name__=='__main__':
    main()

KeyError: 0